In [1]:
#Inisialisasi
import GateLibrary2 as gl
from sympy import Matrix
import sympy as sp
from sympy.physics.quantum import Dagger, TensorProduct
from sympy import simplify,N
import numpy as np

Pertama-tama, kita akan mendefinisikan

$\rho_c = \begin{pmatrix} \gamma & \sqrt{\gamma(1-\gamma)} \\ \sqrt{\gamma(1-\gamma)} & 1-\gamma \end{pmatrix}$

dan juga

$\rho_t = \frac{1}{2}\sigma_0 + \frac{a}{2}\sigma_1 + \frac{b}{2}\sigma_2 + \frac{c}{2}\sigma_3 = \frac{1}{2}\begin{pmatrix}
    1+c & a-ib\\ a+ib & 1-c
\end{pmatrix}$

In [2]:
#Definisikan control qubit
gamma = sp.Symbol('gamma',real = True)
rho_c = Matrix([[gamma,sp.sqrt(gamma*(1-gamma))],[sp.sqrt(gamma*(1-gamma)),1-gamma]])

#Definisikan transmitted qubit
a = sp.Symbol('a',real = True)
b = sp.Symbol('b',real = True)
c = sp.Symbol('c',real = True)
pauli0 = sp.Matrix([[1,0],[0,1]])
pauli1 = sp.Matrix([[0,1],[1,0]])
pauli2 = sp.Matrix([[0,-sp.I],[sp.I,0]])
pauli3 = sp.Matrix([[1,0],[0,-1]])
rho_t = sp.Rational(1/2)*pauli0 + (a/2)*pauli1 + (b/2)*pauli2 + (c/2)*pauli3



Menentukan kekuatan noise dan juga nilai gamma yang diinginkan

In [3]:
#Kekuatan noise
NoiseStrength = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]

#Nilai gamma
GammaValue = 1/2
GammaValue = simplify(GammaValue)

Sekarang kita juga akan mendefinisikan gate yang terdapat pada GateLibrary2.py

In [4]:
gate_p = gl.choose_gate('p',4)
gate_q = gl.choose_gate('p',4)

In [5]:
def Holevo(rho_c,rho_t,gate_p,gate_q,NoiseStrength,GammaValue):
    #Bentuk \rho_c \otimes \rho_t
    total_qubit = TensorProduct(rho_c,rho_t)

    rho0 = Matrix([[1,0],[0,0]])
    rho1 = Matrix([[0,0],[0,1]])
    #Bentuk K_{ij}
    K = [[0 for i in range(4)] for i in range(4)]
    K_dag = [[0 for i in range(4)] for i in range(4)]
    for i in range(4):
        for j in range(4):
            pauli_ij = gate_p['matrices'][i]*gate_q['matrices'][j]
            pauli_ji = gate_p['matrices'][j]*gate_q['matrices'][i]
            K[i][j] = (TensorProduct(rho0,pauli_ij) + TensorProduct(rho1,pauli_ji))
            K_dag[i][j] = Dagger(TensorProduct(rho0,pauli_ij) + TensorProduct(rho1,pauli_ji))

    #Inisialisasi matrix untuk output state
    output_state = Matrix([[0 for i in range(4)] for j in range(4)])

    #Jumlahkan K_{ij} \rho_c\otimes\rho_t \K_{ij}^\dagger
    for i in range(4):
        for j in range(4):
            output_state += K[i][j]*total_qubit*K_dag[i][j]

    #Bagian ini perlu direvisi, kalau kasus keduanya sama strengthnya atau beda seperti apa
    p = sp.Symbol('p',real = True)
    #q = sp.Symbol('q',real = True)
    output_subs = output_state.subs(p,NoiseStrength)
    output_subs2 = output_subs
    #output_subs2 = output_subs.subs(q,1/2)
    #output_state
    output_subs2

    #Bagian ini juga perlu direvisi, lebih baik melakukan partial trace dengan iterasi
    #Cari rho_c_tilde
    rho_c_tilde = Matrix([[output_subs2[0,0]+output_subs2[1,1],output_subs2[0,2]+output_subs2[1,3]],[output_subs2[2,0]+output_subs2[3,1],output_subs2[2,2]+output_subs2[3,3]]])


    #Cari eigen value dari output state dan \tilde{\rho}_c
    Eigen_value = output_subs2.eigenvals()
    EigValC = rho_c_tilde.eigenvals()


    #Bagian ini perlu direvisi, ada cara yang lebih baik untuk memasukkan eigen value ke dalam array
    #Buat eigen value ke dalam array
    keys = [k for k in Eigen_value]
    kunci = 0
    for i in keys:
        if Eigen_value[i]>1:
            kunci = i
    keys.append(kunci)


    keys_c = [k for k in EigValC]
    kunci = 0
    for i in keys_c:
        if EigValC[i]>1:
            kunci = i
    keys_c.append(kunci)

    #Substitusi nilai parameter, nanti harus direvisi agar ditentukan di awal fungsi
    for i in range(4):
        keys[i] = keys[i].subs(a,0)
        keys[i] = keys[i].subs(b,0)
        keys[i] = keys[i].subs(c,1)
        keys[i] = keys[i].subs(gamma,GammaValue)

    for i in range(2):
        keys_c[i] = keys_c[i].subs(a,0)
        keys_c[i] = keys_c[i].subs(b,0)
        keys_c[i] = keys_c[i].subs(c,1)
        keys_c[i] = keys_c[i].subs(gamma,GammaValue)

    #Hitung von Neumann entropy menggunakan nilai eigen
    H_output = gl.vonNeumann(keys)
    #for i in keys:
    #    if i == 0:
    #        H_output = H_output + 0
    #    else:
    #        H_output = H_output + i*sp.log(i)

    H_c = gl.vonNeumann(keys_c)
    #for i in keys_c:
    #    if i == 0:
    #        H_c = H_c + 0
    #    else:
    #        H_c = H_c + i*sp.log(i)

    #Hitung Holevo capacity
    Holevo_cap = 1 + N(H_c) - N(H_output)

    return Holevo_cap




In [6]:
#Sekarang kita akan mencari hasil untuk tiap strength yang berbeda
hasil = [0.0+0j for i in range(len(NoiseStrength))]
for i in range(len(NoiseStrength)):
    hasil[i] = Holevo(rho_c,rho_t,gate_p,gate_q,NoiseStrength[i],GammaValue)

print(hasil)

[1, 0.985370607084091 - 9.018747709474e-17*I, 0.961727660497548 - 2.49457165613233e-17*I, 0.941479071539821 - 6.99434220360957e-18*I, 0.929353753995058 - 1.24617211622026e-18*I, 0.926207656006308 - 3.12837334902237e-19*I, 0.930458930584062 + 4.4603391918839e-19*I, 0.939072722587361 + 6.5687665889403e-20*I, 0.948477319925378 + 3.14773550505079e-19*I, 0.955565923368883 + 8.22038213067711e-19*I, 0.958792180938041]
